<a href="https://www.kaggle.com/code/averma111/pytorch-cafa-5-prediction?scriptVersionId=130939241" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
%%capture 
!pip install torchmetrics

In [ ]:
%%capture
!pip install torchsummary

In [ ]:
## https://www.kaggle.com/code/alexandervc/baseline-multilabel-to-multitarget-binary#Load-train-features---precalculated-embeddings-for-the-proteins
import os
import gc
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
tqdm.pandas()
import torch
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchmetrics import AUROC,F1Score
from torchmetrics.classification import BinaryF1Score
from torchsummary import summary as torchsummary


In [ ]:
def get_dataframe(path):
    return pd.read_csv(path,sep='\t')

In [ ]:
train_terms = '/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv'
train_taxonomy ='/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv'

In [ ]:
get_dataframe(train_terms).head()

In [ ]:
get_dataframe(train_taxonomy).head()


In [ ]:
def summary(text, df):
    print(f'{text} shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summ['null'] = df.isnull().sum()
    summ['unique'] = df.nunique()
    summ['min'] = df.min()
    summ['median'] = df.median()
    summ['max'] = df.max()
    summ['mean'] = df.mean()
    summ['std'] = df.std()
    summ['duplicate'] = df.duplicated().sum()
    return summ

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
summary('train_terms',reduce_mem_usage(get_dataframe(train_terms)))

In [ ]:
summary('train_terms',reduce_mem_usage(get_dataframe(train_taxonomy)))

In [ ]:
sns.countplot(data=reduce_mem_usage(get_dataframe(train_terms)),x='aspect',color='r')

In [ ]:
train_terms=reduce_mem_usage(get_dataframe(train_terms))

In [ ]:
def get_train_dataset():
    train_protein_ids = np.load('/kaggle/input/t5embeds/train_ids.npy')
    train_embeddings = np.load('/kaggle/input/t5embeds/train_embeds.npy')
    column_num = train_embeddings.shape[1]
    train = pd.DataFrame(train_embeddings, columns = ["Column_" + str(i) for i in range(1, column_num+1)])
    return train,train_protein_ids

train,train_protein_ids = get_train_dataset()
print(train.shape,train_protein_ids.shape)

In [ ]:
num_of_labels = 1500
def get_label_train_terms(df):
    labels=df['term'].value_counts().index[:num_of_labels].tolist()
    train_terms_updated=df.loc[df['term'].isin(labels)]
    return labels,train_terms_updated

labels_count,train_terms_updated=get_label_train_terms(train_terms)

In [ ]:
def show_pit_aspects():
    pie_df = train_terms_updated['aspect'].value_counts()
    palette_color = sns.color_palette('pastel')
    plt.pie(pie_df.values, labels=np.array(pie_df.index), colors=palette_color, autopct='%.0f%%')
    plt.show()
    
show_pit_aspects()

In [ ]:
def get_labels(train_protein_ids):
    train_size = train_protein_ids.shape[0] # len(X)
    train_labels = np.zeros((train_size ,num_of_labels))
    series_train_protein_ids = pd.Series(train_protein_ids)

    for i in range(num_of_labels):
        n_train_terms = train_terms_updated[train_terms_updated['term'] ==  labels_count[i]]
        label_related_proteins = n_train_terms['EntryID'].unique()
        train_labels[:,i] =  series_train_protein_ids.isin(label_related_proteins).astype(float)
    return train_labels


train_labels=get_labels(train_protein_ids)

labels = pd.DataFrame(data = train_labels, columns = labels_count)
print(labels.shape)

In [ ]:
def train_test_dataset(features,labels):
    return  train_test_split(features,labels,shuffle=True,random_state=42)

X_train,X_val,y_train,y_val = train_test_dataset(train,labels)
print(X_train.shape,X_val.shape,y_train.shape,y_val.shape)



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
class FluxData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
            return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

X_data = torch.from_numpy(X_train.values).float().to(device)
y_data = torch.from_numpy(y_train.values).float().to(device)
X_val = torch.from_numpy(X_val.values).float().to(device)
y_val = torch.from_numpy(y_val.values).float().to(device)
train_data = FluxData(X_data,y_data)
test_data = FluxData(X_val,y_val)

In [ ]:
class CAFA5NNetBase(torch.nn.Module):
    
    def training_step(self,batch):
        features,labels = batch
        out = self(features)
        loss = F.binary_cross_entropy(out,labels)
        return loss
    
    def validation_step(self, batch):
        features, labels = batch 
        out = self(features)                    # Generate predictions
        loss = F.binary_cross_entropy(out, labels)   # Calculate loss
        acc = auroc(out, labels)           # Calculate accuracy
        return {'Validation_loss': loss.detach(), 'Validation_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['Validation_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['Validation_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'Validation_loss': epoch_loss.item(), 'Validation_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        if epoch%5==0:
            print("Epoch [{}], Train_loss: {:.4f}, Validation_loss: {:.4f}, Validation_acc: {:.4f}".format(
            epoch, result['Train_loss'], result['Validation_loss'], result['Validation_acc']))

In [ ]:
class CAFA5NNet(CAFA5NNetBase):
    def __init__(self,input_features,output_features):
        super(CAFA5NNet,self).__init__()
        self.network = torch.nn.Sequential(
        torch.nn.Linear(input_features,512),
        torch.nn.ReLU(),
        torch.nn.Linear(512,512),    
        torch.nn.ReLU(),
        torch.nn.Linear(512,output_features),
        torch.nn.Sigmoid()
        )
    def forward(self,inputs):
        return self.network(inputs)

In [ ]:
model = CAFA5NNet(X_train.shape[1],y_train.shape[1])
model.to(device)

In [ ]:
torchsummary(model, X_data.size(), batch_size=-1, device='cuda')

In [ ]:
BATCH_SIZE = 5120
EPOCHS = 50
LEARNING_RATE = 0.001
MOMENTUM = 0.9
OPT_FUNC = torch.optim.Adam

In [ ]:
def get_dataloaders(dataset_type,batch,shuffle):
    if shuffle:
         return DataLoader(dataset=dataset_type, batch_size=batch, shuffle=True)
    else:
        return DataLoader(dataset=dataset_type, batch_size=batch,shuffle=False)
    
train_dl = get_dataloaders(train_data,BATCH_SIZE,True)
val_dl = get_dataloaders(test_data,BATCH_SIZE,False)

In [ ]:
def auroc(outputs, labels):
    auroc = AUROC(task="binary")
    return auroc(outputs, labels)

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = OPT_FUNC):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in tqdm(range(epochs)):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['Train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

In [ ]:
history = fit(EPOCHS, LEARNING_RATE, model, train_dl, val_dl,OPT_FUNC)

In [ ]:
def plot_accuracies(history):
    """ Plot the history of accuracies"""
    accuracies = [x['Validation_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs. No. of epochs');
    

plot_accuracies(history)

In [ ]:
def plot_losses(history):
    """ Plot the losses in each epoch"""
    train_losses = [x.get('Train_loss') for x in history]
    val_losses = [x['Validation_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

plot_losses(history)

In [ ]:
def get_test_dataset():
    test_embeddings = np.load('/kaggle/input/t5embeds/test_embeds.npy')
    column_num = test_embeddings.shape[1]
    test = pd.DataFrame(test_embeddings, columns = ["Column_" + str(i) for i in range(1, column_num+1)])
    return test 

test = get_test_dataset()
print(test.shape)

In [ ]:
class CAFA5TestData(Dataset):
    
    def __init__(self, X_test_data):
        self.X_test_data = X_test_data
        
    def __getitem__(self, index):
        return self.X_test_data[index]
        
    def __len__ (self):
        return len(self.X_test_data)

In [ ]:
test_data = CAFA5TestData(torch.from_numpy(test.values).float().to(device))
test_data_loader = DataLoader(dataset=test_data, batch_size=test.shape[0])


In [ ]:

def eval_test_data(model,testing_data_dl):
    model.eval()
    with torch.no_grad():
        for X_batch_test in testing_data_dl:
            X_batch_test = X_batch_test.to(device)
            predictions = model(X_batch_test)
            prediction_target=predictions.detach().numpy()

    return prediction_target

prediction_target = eval_test_data(model,test_data_loader)

In [ ]:
prediction_target.shape

In [ ]:
def make_predictions(prediction_target):
    test_protein_ids = np.load('/kaggle/input/t5embeds/test_ids.npy')
    protein_list = []
    for k in list(test_protein_ids):
        protein_list += [k] * prediction_target.shape[1] 
    return protein_list

protein_list=make_predictions(prediction_target)      

In [ ]:
def submit():
    df_submission = pd.DataFrame(columns = ['Protein Id', 'GO Term Id','Prediction'])
    df_submission['Protein Id'] = protein_list
    df_submission['GO Term Id'] = labels_count * prediction_target.shape[0]
    df_submission['Prediction'] = prediction_target.ravel()
    df_submission.to_csv("submission.tsv",header=False, index=False,sep='\t')
    
submit()

In [ ]:
temp=pd.read_csv('/kaggle/working/submission.tsv',sep='\t')
temp.count()